In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3 as sql

# Data

In [5]:
flights = [
    {"time_hour": "2024-11-26 08:00:00", "origin": "JFK", "flight": 1, "dest": "LAX", "tailnum": "N12345", "carrier": "AA"},
    {"time_hour": "2024-11-26 09:00:00", "origin": "LGA", "flight": 2, "dest": "ORD", "tailnum": "N67890", "carrier": "UA"},
    {"time_hour": "2024-11-26 10:00:00", "origin": "EWR", "flight": 3, "dest": "MIA", "tailnum": "N54321", "carrier": "DL"}
]
airports = [
    {"faa": "JFK", "name": "John F Kennedy Intl", "lat": 40.639751, "lon": -73.778925, "alt": 13, "tz": -5, "dst": "A", "city": "New York"},
    {"faa": "LGA", "name": "La Guardia", "lat": 40.777245, "lon": -73.872608, "alt": 22, "tz": -5, "dst": "A", "city": "New York"},
    {"faa": "EWR", "name": "Newark Liberty Intl", "lat": 40.6925, "lon": -74.168667, "alt": 18, "tz": -5, "dst": "A", "city": "Newark"}
]
weather = [
    {"time_hour": "2024-11-26 08:00:00", "origin": "JFK", "temp": 12.5, "humidity": 50, "wind_speed": 10},
    {"time_hour": "2024-11-26 09:00:00", "origin": "LGA", "temp": 10.0, "humidity": 55, "wind_speed": 12},
    {"time_hour": "2024-11-26 10:00:00", "origin": "EWR", "temp": 15.2, "humidity": 48, "wind_speed": 8}
]
planes = [
    {"tailnum": "N12345", "type": "Boeing 737", "manufacturer": "Boeing", "model": "737-800", "seats": 160, "year": 2010},
    {"tailnum": "N67890", "type": "Airbus A320", "manufacturer": "Airbus", "model": "A320-200", "seats": 150, "year": 2012},
    {"tailnum": "N54321", "type": "Boeing 757", "manufacturer": "Boeing", "model": "757-200", "seats": 200, "year": 2005}
]
airlines = [
    {"carrier": "AA", "name": "American Airlines"},
    {"carrier": "UA", "name": "United Airlines"},
    {"carrier": "DL", "name": "Delta Air Lines"}
]
passengers_data = [
    {"passenger_id": 1, "name": "Alice", "flight": 1},
    {"passenger_id": 2, "name": "Bob", "flight": 1},
    {"passenger_id": 3, "name": "Charlie", "flight": 2},
    {"passenger_id": 4, "name": "Daisy", "flight": 3},
    {"passenger_id": 5, "name": "Eve", "flight": 3},
]

ticket_prices_data = [
    {"flight": 1, "price": 200.0},
    {"flight": 2, "price": 180.0},
    {"flight": 3, "price": 220.0},
    {"flight": 4, "price": 150.0},  # A flight ID not present in `flights`
]

# Joining

<img src="https://www.alphacodingskills.com/sql/img/sql-join.PNG" width="500px"/>

In [6]:
flights_df = pd.DataFrame(flights)
flights_df

,time_hour,origin,flight,dest,tailnum,carrier
0,2024-11-26 08:00:00,JFK,1,LAX,N12345,AA
1,2024-11-26 09:00:00,LGA,2,ORD,N67890,UA
2,2024-11-26 10:00:00,EWR,3,MIA,N54321,DL


In [7]:
passengers_df = pd.DataFrame(passengers_data)
passengers_df

,passenger_id,name,flight
0,1,Alice,1
1,2,Bob,1
2,3,Charlie,2
3,4,Daisy,3
4,5,Eve,3


In [8]:
prices_df = pd.DataFrame(ticket_prices_data)
prices_df

,flight,price
0,1,200.0
1,2,180.0
2,3,220.0
3,4,150.0


In [9]:
# Join passengers to flights
passengers_df.merge(flights_df, on="flight", how="inner")

,passenger_id,name,flight,time_hour,origin,dest,tailnum,carrier
0,1,Alice,1,2024-11-26 08:00:00,JFK,LAX,N12345,AA
1,2,Bob,1,2024-11-26 08:00:00,JFK,LAX,N12345,AA
2,3,Charlie,2,2024-11-26 09:00:00,LGA,ORD,N67890,UA
3,4,Daisy,3,2024-11-26 10:00:00,EWR,MIA,N54321,DL
4,5,Eve,3,2024-11-26 10:00:00,EWR,MIA,N54321,DL


In [10]:
# Join flights and prices
flights_df.merge(prices_df, on="flight", how="left")

,time_hour,origin,flight,dest,tailnum,carrier,price
0,2024-11-26 08:00:00,JFK,1,LAX,N12345,AA,200.0
1,2024-11-26 09:00:00,LGA,2,ORD,N67890,UA,180.0
2,2024-11-26 10:00:00,EWR,3,MIA,N54321,DL,220.0


In [11]:
# Join all to large df
passengers_df \
    .merge(flights_df, on="flight", how="inner") \
    .merge(prices_df, on="flight", how="inner")

,passenger_id,name,flight,time_hour,origin,dest,tailnum,carrier,price
0,1,Alice,1,2024-11-26 08:00:00,JFK,LAX,N12345,AA,200.0
1,2,Bob,1,2024-11-26 08:00:00,JFK,LAX,N12345,AA,200.0
2,3,Charlie,2,2024-11-26 09:00:00,LGA,ORD,N67890,UA,180.0
3,4,Daisy,3,2024-11-26 10:00:00,EWR,MIA,N54321,DL,220.0
4,5,Eve,3,2024-11-26 10:00:00,EWR,MIA,N54321,DL,220.0


# SQL

<img src="https://r4ds.hadley.nz/diagrams/relational.png" width="500px"/>

## Querying SQL database

In [12]:
conn = sql.connect("./flights.sqlite3")
cursor = conn.cursor()

In [13]:
tables = list(cursor.execute("SELECT name FROM sqlite_master WHERE type='table';"))
tables

[('airports',), ('flights',), ('weather',), ('planes',), ('airlines',)]

In [20]:
# SELECT

query = """
SELECT * 
FROM flights
"""

cursor.execute(query)

for data in cursor.fetchall():
    print(data)
pd.read_sql_query(query, conn)

('2024-11-26 08:00:00', 'JFK', 1, 'LAX', 'N12345', 'AA')
('2024-11-26 09:00:00', 'LGA', 2, 'ORD', 'N67890', 'UA')
('2024-11-26 10:00:00', 'EWR', 3, 'MIA', 'N54321', 'DL')


,time_hour,origin,flight,dest,tailnum,carrier
0,2024-11-26 08:00:00,JFK,1,LAX,N12345,AA
1,2024-11-26 09:00:00,LGA,2,ORD,N67890,UA
2,2024-11-26 10:00:00,EWR,3,MIA,N54321,DL


In [27]:
# FILTER
query = """
SELECT * 
FROM flights
WHERE flight > 1 
AND origin="LGA"
"""

pd.read_sql_query(query, conn)

,time_hour,origin,flight,dest,tailnum,carrier
0,2024-11-26 09:00:00,LGA,2,ORD,N67890,UA


In [29]:
# JOIN
query = """
SELECT origin, flight, dest, flights.carrier, name
FROM flights
LEFT JOIN airlines ON flights.carrier = airlines.carrier;
"""

pd.read_sql_query(query, conn)

,origin,flight,dest,carrier,name
0,JFK,1,LAX,AA,American Airlines
1,LGA,2,ORD,UA,United Airlines
2,EWR,3,MIA,DL,Delta Air Lines


## Insert data into Sqlite3 db file

In [21]:
# Connect to SQLite database (creates the file if it doesn't exist)
conn = sqlite3.connect("flights.sqlite3")
cursor = conn.cursor()

# Create tables
cursor.execute("""
CREATE TABLE airports (
    faa TEXT PRIMARY KEY,
    name TEXT,
    lat REAL,
    lon REAL,
    alt INTEGER,
    tz INTEGER,
    dst TEXT,
    city TEXT
)
""")
cursor.execute("""
CREATE TABLE flights (
    time_hour TEXT,
    origin TEXT,
    flight INTEGER,
    dest TEXT,
    tailnum TEXT,
    carrier TEXT,
    PRIMARY KEY (time_hour, flight)
)
""")
cursor.execute("""
CREATE TABLE weather (
    time_hour TEXT,
    origin TEXT,
    temp REAL,
    humidity INTEGER,
    wind_speed REAL,
    PRIMARY KEY (time_hour, origin)
)
""")
cursor.execute("""
CREATE TABLE planes (
    tailnum TEXT PRIMARY KEY,
    type TEXT,
    manufacturer TEXT,
    model TEXT,
    seats INTEGER,
    year INTEGER
)
""")
cursor.execute("""
CREATE TABLE airlines (
    carrier TEXT PRIMARY KEY,
    name TEXT
)
""")

# Insert data into tables
# Airports
cursor.executemany("""
INSERT INTO airports (faa, name, lat, lon, alt, tz, dst, city)
VALUES (:faa, :name, :lat, :lon, :alt, :tz, :dst, :city)
""", airports)

# Flights
cursor.executemany("""
INSERT INTO flights (time_hour, origin, flight, dest, tailnum, carrier)
VALUES (:time_hour, :origin, :flight, :dest, :tailnum, :carrier)
""", flights)

# Weather
cursor.executemany("""
INSERT INTO weather (time_hour, origin, temp, humidity, wind_speed)
VALUES (:time_hour, :origin, :temp, :humidity, :wind_speed)
""", weather)

# Planes
cursor.executemany("""
INSERT INTO planes (tailnum, type, manufacturer, model, seats, year)
VALUES (:tailnum, :type, :manufacturer, :model, :seats, :year)
""", planes)

# Airlines
cursor.executemany("""
INSERT INTO airlines (carrier, name)
VALUES (:carrier, :name)
""", airlines)

# Commit and close connection
conn.commit()
conn.close()

# RegEx

In [30]:
import re

In [46]:
string = "Please contact support@example.com for assistance. For general inquiries, you can email info@company.com and @gmail.com"
print(string)

Please contact support@example.com for assistance. For general inquiries, you can email info@company.com and @gmail.com


In [57]:
pattern = r"\b[\w._%+-]+@[\w.-]+\.[A-Za-z]{2,4}\b"

r"\b[\w._%+-]+@[\w.-]+\.[A-Za-z]{2,4} \b"

'\\b [\\w._%+-]*  \\b'

In [58]:
emails = re.findall(pattern, string)
print(emails)

['support@example.com', 'info@company.com']
